# Labo 07 

#### Jarod Streckeisen - Dimitri De Bleser

## Préparation des données

In [1]:
import nltk
from nltk.corpus import reuters
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Charger les données Reuters
documents = reuters.fileids()
categories = ['money-fx', 'grain', 'nat-gas']

# Fonction pour extraire le texte des documents
def get_text(doc_id):
    return reuters.raw(doc_id)

# Séparer les données en train et test comme dans NLTK
train_docs_id = list(filter(lambda doc: doc.startswith("training"), documents))
test_docs_id = list(filter(lambda doc: doc.startswith("test"), documents))

train_docs = [get_text(doc_id) for doc_id in train_docs_id]
test_docs = [get_text(doc_id) for doc_id in test_docs_id]

## Classifieurs  binaires

#### Choix des hyperparametres test: 
- Suppression des stopwords → {oui, non} 
- Représentation des documents → {Bernoulli, multinomiale} 

In [2]:
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd

# Fonction pour obtenir les étiquettes binaires pour une catégorie donnée
def get_labels(docs_id, category):
    return [1 if category in reuters.categories(doc_id) else 0 for doc_id in docs_id]

# Création des étiquettes binaires pour chaque catégorie
train_labels = {category: get_labels(train_docs_id, category) for category in categories}
test_labels = {category: get_labels(test_docs_id, category) for category in categories}

# Définir une fonction pour entraîner et valider les modèles
def train_and_evaluate(train_docs, train_labels, test_docs, test_labels, use_stopwords, model_type):
    # Vectorisation des textes
    stop_words = 'english' if use_stopwords else None
    vectorizer = CountVectorizer(stop_words=stop_words, binary=(model_type == 'Bernoulli'))
    X_train = vectorizer.fit_transform(train_docs)
    X_test = vectorizer.transform(test_docs)
    
    # Choisir le classifieur
    if model_type == 'Multinomial':
        model = MultinomialNB()
    elif model_type == 'Bernoulli':
        model = BernoulliNB()
    
    model.fit(X_train, train_labels)
    predictions = model.predict(X_test)
    
    precision = precision_score(test_labels, predictions, zero_division=0)
    recall = recall_score(test_labels, predictions)
    f1 = f1_score(test_labels, predictions)
    
    return precision, recall, f1

# Validation croisée pour chaque catégorie
best_params = {}
for category in categories:
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
    train_idx, dev_idx = next(sss.split(train_docs, train_labels[category]))
    train_docs_cv, dev_docs_cv = np.array(train_docs)[train_idx], np.array(train_docs)[dev_idx]
    train_labels_cv, dev_labels_cv = np.array(train_labels[category])[train_idx], np.array(train_labels[category])[dev_idx]
    
    best_f1 = 0
    for use_stopwords in [True, False]:
        for model_type in ['Multinomial', 'Bernoulli']:
            precision, recall, f1 = train_and_evaluate(train_docs_cv, train_labels_cv, dev_docs_cv, dev_labels_cv, use_stopwords, model_type)
            if f1 > best_f1:
                best_f1 = f1
                best_params[category] = {'use_stopwords': use_stopwords, 'model_type': model_type}

# Entraînement final avec les meilleurs hyperparamètres
final_results = {}
for category in categories:
    params = best_params[category]
    precision, recall, f1 = train_and_evaluate(train_docs, train_labels[category], test_docs, test_labels[category], params['use_stopwords'], params['model_type'])
    final_results[category] = {'precision': precision, 'recall': recall, 'f1': f1}
    
# Création du DataFrame pandas
best_params_df = pd.DataFrame(best_params)
scores_df_binaire = pd.DataFrame(final_results).round(2)

# Affichage du tableau
print(" -- Meilleurs hyperparamètres --")
print(best_params_df)
print("")
print(" -- Scores Binaire --")
print(scores_df_binaire)

 -- Meilleurs hyperparamètres --
                  money-fx        grain      nat-gas
use_stopwords        False         True         True
model_type     Multinomial  Multinomial  Multinomial

 -- Scores Binaire --
           money-fx  grain  nat-gas
precision      0.58   0.57     0.55
recall         0.98   0.95     0.40
f1             0.73   0.71     0.46


## Classifieur multi-classe

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Fonction pour obtenir l'étiquette principale pour chaque document
def get_primary_label(doc_id):
    doc_categories = reuters.categories(doc_id)
    for category in categories:
        if category in doc_categories:
            return category
    return 'other'

# Créer les étiquettes pour l'entraînement et le test
train_primary_labels = [get_primary_label(doc_id) for doc_id in train_docs_id]
test_primary_labels = [get_primary_label(doc_id) for doc_id in test_docs_id]

# Encodage des étiquettes en entiers
label_encoder = LabelEncoder()
train_primary_labels_encoded = label_encoder.fit_transform(train_primary_labels)
test_primary_labels_encoded = label_encoder.transform(test_primary_labels)

# Choix des meilleurs hyperparamètres des classifieurs binaires
use_stopwords = True  # basé sur les observations précédentes
model_type = 'Multinomial'  # basé sur les observations précédentes

# Vectorisation des textes
stop_words = 'english' if use_stopwords else None
vectorizer = CountVectorizer(stop_words=stop_words, binary=(model_type == model_type))
X_train = vectorizer.fit_transform(train_docs)
X_test = vectorizer.transform(test_docs)

# Entraînement du modèle multi-classe
multi_class_model = LogisticRegression(max_iter=1000, random_state=0)
multi_class_model.fit(X_train, train_primary_labels_encoded)

# Prédiction sur le test set
predictions_multi_class = multi_class_model.predict(X_test)

In [4]:
# Obtenir le rapport de classification
report = classification_report(test_primary_labels_encoded, predictions_multi_class, target_names=label_encoder.classes_, output_dict=True)

# Création du DataFrame pandas
scores_df = pd.DataFrame(report).round(2)

# Affichage du tableau
print(" -- Scores multiclasse --")
print(scores_df)
print("")
print(" -- Scores Binaire --")
print(scores_df_binaire)

 -- Scores multiclasse --
            grain  money-fx  nat-gas    other  accuracy  macro avg  \
precision    0.91      0.79     0.71     0.96      0.95       0.84   
recall       0.80      0.69     0.40     0.98      0.95       0.72   
f1-score     0.85      0.73     0.51     0.97      0.95       0.77   
support    148.00    179.00    30.00  2662.00      0.95    3019.00   

           weighted avg  
precision          0.95  
recall             0.95  
f1-score           0.95  
support         3019.00  

 -- Scores Binaire --
           money-fx  grain  nat-gas
precision      0.58   0.57     0.55
recall         0.98   0.95     0.40
f1             0.73   0.71     0.46


### Conclusion
On peut voir que les scores avec un model multi-classe est superieur au models binaire. La classification multi-classe semble
donc etre la bonne strategie. Il est a notee que la classe nat-gas a un score bas, qui peut s'expliquer par une basse representation de la classe dans les donnees.